In [2]:
!pip install torch_geometric

  Using cached torch_geometric-2.5.3-py3-none-any.whl.metadata (64 kB)
Using cached torch_geometric-2.5.3-py3-none-any.whl (1.1 MB)

[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 514.7 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1
    Uninstalling pip-24.1:
      Successfully uninstalled pip-24.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 329.4 kB/s eta 0:00:0000:0100:03


In [5]:
!pip install torch torchvision torchaudio

In [6]:
!pip install torch-geometric torch-scatter torch-sparse
from torch_geometric.nn import global_mean_pool

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp311-cp311-macosx_11_0_arm64.whl size=270958 sha256=8a1847c2d7d0fbe9dfb9b2dc2a33ae0bd31afa92bdc44a1462331b674ef82737
  Stored in directory: /Users/nonzuzosikhosana/Library/Caches/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp311-cp311-macosx_11_0_arm64.whl size=457778 sha256=20e3c668d56b2fd7e396cb5be650a8bb8bf5402f3b382172fe3108d85a5ff67a
  Stored in directory: /Users/nonzuzosikhosana/Library/Caches/pip/wheels/75/e2/1e/299c596063839303657c211f587f05591891cc6cf126d94d21
Successfully built torch-scatter torch-sparse


In [ ]:
import torch
from torch_geometric.data import Data, DataLoader, Batch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from rdkit import Chem
import pandas as pd
from sklearn.model_selection import train_test_split


In [53]:
import numpy as np
import pandas as pd

In [54]:
df = pd.read_csv("PubChem_compound_text_paracetamol.csv", na_values = '-')

In [55]:
df.describe()

,cid,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,...,definedatomstereocnt,undefinedatomstereocnt,totalbondstereocnt,definedbondstereocnt,undefinedbondstereocnt,pclidcnt,gpidcnt,gpfamilycnt,annothitcnt,cidcdate
count,9.900000e+01,99.000000,99.000000,99.000000,65.000000,99.000000,99.000000,99.00000,99.000000,99.00000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,9.900000e+01
mean,5.408764e+07,293.440303,96.547475,319.130303,0.733846,20.515152,2.878788,4.79798,3.464646,293.20801,...,0.636364,0.121212,0.030303,0.020202,0.010101,1037.525253,4139.070707,1600.030303,3.929293,2.010565e+07
std,5.990447e+07,156.142455,50.917727,215.880771,1.507925,11.160575,1.842097,2.77371,2.918320,156.03098,...,1.587287,0.558291,0.172292,0.141407,0.100504,7493.317599,19550.718228,7485.042052,4.488654,6.151277e+04
min,4.030000e+02,109.130000,29.100000,66.900000,-3.800000,8.000000,0.000000,1.00000,0.000000,109.05300,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.004092e+07
25%,1.462005e+05,182.155000,52.400000,156.500000,0.200000,12.500000,2.000000,3.00000,1.000000,182.04550,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.005081e+07
50%,2.282192e+07,265.420000,87.000000,308.000000,0.500000,18.000000,2.000000,4.00000,3.000000,265.15000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,3.000000,2.007120e+07
75%,1.296913e+08,339.785000,138.000000,394.500000,1.400000,23.000000,4.000000,6.00000,4.000000,339.59550,...,0.000000,0.000000,0.000000,0.000000,0.000000,6.500000,47.500000,17.500000,5.000000,2.017091e+07
max,1.694523e+08,1034.500000,240.000000,1200.000000,8.000000,72.000000,9.000000,15.00000,19.000000,1033.41000,...,8.000000,5.000000,1.000000,1.000000,1.000000,72373.000000,124556.000000,47229.000000,20.000000,2.023122e+07


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0    cid                    99 non-null     int64  
 1   cmpdname                99 non-null     object 
 2   cmpdsynonym             99 non-null     object 
 3   mw                      99 non-null     float64
 4   mf                      99 non-null     object 
 5   polararea               99 non-null     float64
 6   complexity              99 non-null     float64
 7   xlogp                   65 non-null     float64
 8   heavycnt                99 non-null     int64  
 9   hbonddonor              99 non-null     int64  
 10  hbondacc                99 non-null     int64  
 11  rotbonds                99 non-null     int64  
 12  inchi                   99 non-null     object 
 13  isosmiles               99 non-null     object 
 14  canonicalsmiles         99 non-null     obje

## Hnadling Missing Values

In [57]:
##Drop unnecessary columns 
df.drop([' cid', 'cidcdate', 'annotation'], inplace=True, axis=1)


In [62]:
df.dropna(inplace=True, axis = 0)


## Splitting the Data into Numerical and Categorical 

In [58]:
 #separating numerical and non-numerical columns
numeric=df.select_dtypes(include=np.number)

numeric.head()


,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,monoisotopicmass,...,totalatomstereocnt,definedatomstereocnt,undefinedatomstereocnt,totalbondstereocnt,definedbondstereocnt,undefinedbondstereocnt,pclidcnt,gpidcnt,gpfamilycnt,annothitcnt
0,139.11,66.0,123.0,1.9,10,1,3,0,139.027,139.027,...,0,0,0,0,0,0,18008,86483,30288,20
1,109.13,46.2,66.9,0.0,8,2,2,0,109.053,109.053,...,0,0,0,0,0,0,5040,115629,47229,19
2,135.16,29.1,116.0,1.2,10,1,1,1,135.068,135.068,...,0,0,0,0,0,0,4489,49112,21559,19
3,151.16,49.3,139.0,0.5,11,2,2,1,151.063,151.063,...,0,0,0,0,0,0,72373,124556,45423,18
4,136.15,37.3,123.0,1.4,10,1,2,1,136.052,136.052,...,0,0,0,0,0,0,999,17627,7654,15


In [59]:
# Example: Convert all non-numeric columns to object type
non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
non_numeric = df[non_numeric_columns].astype('object')
non_numeric.head()

,cmpdname,cmpdsynonym,mf,inchi,isosmiles,canonicalsmiles,inchikey,iupacname,neighbortype,meshheadings,annothits,aids,sidsrcname,depcatg
0,4-Nitrophenol,"4-Nitrophenol|p-nitrophenol|100-02-7|Phenol, 4...",C6H5NO3,"InChI=1S/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H",C1=CC(=CC=C1[N+](=O)[O-])O,C1=CC(=CC=C1[N+](=O)[O-])O,BTJIUGUIPKRLHP-UHFFFAOYSA-N,4-nitrophenol,2D+3D,NaN,Agrochemical Information|Biological Test Resul...,155|157|161|165|167|175|248|1188|15680|19254|1...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...
1,p-Aminophenol,4-aminophenol|123-30-8|p-aminophenol|4-hydroxy...,C6H7NO,"InChI=1S/C6H7NO/c7-5-1-3-6(8)4-2-5/h1-4,8H,7H2",C1=CC(=CC=C1N)O,C1=CC(=CC=C1N)O,PLIKAWJENQZMHA-UHFFFAOYSA-N,4-aminophenol,2D+3D,NaN,Agrochemical Information|Biological Test Resul...,155|157|161|165|167|175|228|248|250|270|328|33...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...
2,Acetanilide,acetanilide|N-Phenylacetamide|103-84-4|Acetami...,C8H9NO,"InChI=1S/C8H9NO/c1-7(10)9-8-5-3-2-4-6-8/h2-6H,...",CC(=O)NC1=CC=CC=C1,CC(=O)NC1=CC=CC=C1,FZERHIULMFGESH-UHFFFAOYSA-N,N-phenylacetamide,2D+3D,NaN,Biological Test Results|Interactions and Pathw...,155|157|161|165|167|175|179|421|426|427|433|43...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...
3,Acetaminophen,acetaminophen|Paracetamol|4-Acetamidophenol|10...,C8H9NO2,InChI=1S/C8H9NO2/c1-6(10)9-7-2-4-8(11)5-3-7/h2...,CC(=O)NC1=CC=C(C=C1)O,CC(=O)NC1=CC=C(C=C1)O,RZVAJINKPMORJF-UHFFFAOYSA-N,N-(4-hydroxyphenyl)acetamide,2D+3D,Acetaminophen,Agrochemical Information|Biological Test Resul...,155|157|161|165|167|175|248|357|377|410|411|42...,10X CHEM|3B Scientific (Wuhan) Corp|3WAY PHARM...,Chemical Vendors|Curation Efforts|Governmental...
4,4'-Hydroxyacetophenone,4'-Hydroxyacetophenone|99-93-4|1-(4-Hydroxyphe...,C8H8O2,"InChI=1S/C8H8O2/c1-6(9)7-2-4-8(10)5-3-7/h2-5,1...",CC(=O)C1=CC=C(C=C1)O,CC(=O)C1=CC=C(C=C1)O,TXFPEBPIARQUIG-UHFFFAOYSA-N,1-(4-hydroxyphenyl)ethanone,2D+3D,NaN,Biological Test Results|Interactions and Pathw...,155|157|161|165|167|175|248|1576|1811|260086|2...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...


## Imputation

In [60]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer


#Training Dataset Numeric
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on the numeric data and transform it
imputed_numeric = imputer.fit_transform(numeric)

# Create a DataFrame from the imputed numeric data
numeric_data = pd.DataFrame(imputed_numeric, columns=numeric.columns)

numeric_data

,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,monoisotopicmass,...,totalatomstereocnt,definedatomstereocnt,undefinedatomstereocnt,totalbondstereocnt,definedbondstereocnt,undefinedbondstereocnt,pclidcnt,gpidcnt,gpfamilycnt,annothitcnt
0,139.11,66.0,123.0,1.900000,10.0,1.0,3.0,0.0,139.027,139.027,...,0.0,0.0,0.0,0.0,0.0,0.0,18008.0,86483.0,30288.0,20.0
1,109.13,46.2,66.9,0.000000,8.0,2.0,2.0,0.0,109.053,109.053,...,0.0,0.0,0.0,0.0,0.0,0.0,5040.0,115629.0,47229.0,19.0
2,135.16,29.1,116.0,1.200000,10.0,1.0,1.0,1.0,135.068,135.068,...,0.0,0.0,0.0,0.0,0.0,0.0,4489.0,49112.0,21559.0,19.0
3,151.16,49.3,139.0,0.500000,11.0,2.0,2.0,1.0,151.063,151.063,...,0.0,0.0,0.0,0.0,0.0,0.0,72373.0,124556.0,45423.0,18.0
4,136.15,37.3,123.0,1.400000,10.0,1.0,2.0,1.0,136.052,136.052,...,0.0,0.0,0.0,0.0,0.0,0.0,999.0,17627.0,7654.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,356.31,147.0,414.0,0.733846,23.0,2.0,6.0,6.0,356.042,356.042,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,549.60,209.0,141.0,0.733846,39.0,9.0,9.0,3.0,549.269,549.269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,330.30,146.0,437.0,-0.800000,23.0,5.0,8.0,4.0,330.114,330.114,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,317.37,141.0,404.0,0.200000,21.0,4.0,6.0,6.0,317.109,317.109,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
 # Now concatenate numeric_data and non_numeric
df = pd.concat([numeric_data, non_numeric], axis=1)

In [52]:
df.head()

,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,monoisotopicmass,...,isosmiles,canonicalsmiles,inchikey,iupacname,neighbortype,meshheadings,annothits,aids,sidsrcname,depcatg
0,139.11,66.0,123.0,1.9,10.0,1.0,3.0,0.0,139.027,139.027,...,C1=CC(=CC=C1[N+](=O)[O-])O,C1=CC(=CC=C1[N+](=O)[O-])O,BTJIUGUIPKRLHP-UHFFFAOYSA-N,4-nitrophenol,2D+3D,NaN,Agrochemical Information|Biological Test Resul...,155|157|161|165|167|175|248|1188|15680|19254|1...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...
1,109.13,46.2,66.9,0.0,8.0,2.0,2.0,0.0,109.053,109.053,...,C1=CC(=CC=C1N)O,C1=CC(=CC=C1N)O,PLIKAWJENQZMHA-UHFFFAOYSA-N,4-aminophenol,2D+3D,NaN,Agrochemical Information|Biological Test Resul...,155|157|161|165|167|175|228|248|250|270|328|33...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...
2,135.16,29.1,116.0,1.2,10.0,1.0,1.0,1.0,135.068,135.068,...,CC(=O)NC1=CC=CC=C1,CC(=O)NC1=CC=CC=C1,FZERHIULMFGESH-UHFFFAOYSA-N,N-phenylacetamide,2D+3D,NaN,Biological Test Results|Interactions and Pathw...,155|157|161|165|167|175|179|421|426|427|433|43...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...
3,151.16,49.3,139.0,0.5,11.0,2.0,2.0,1.0,151.063,151.063,...,CC(=O)NC1=CC=C(C=C1)O,CC(=O)NC1=CC=C(C=C1)O,RZVAJINKPMORJF-UHFFFAOYSA-N,N-(4-hydroxyphenyl)acetamide,2D+3D,Acetaminophen,Agrochemical Information|Biological Test Resul...,155|157|161|165|167|175|248|357|377|410|411|42...,10X CHEM|3B Scientific (Wuhan) Corp|3WAY PHARM...,Chemical Vendors|Curation Efforts|Governmental...
4,136.15,37.3,123.0,1.4,10.0,1.0,2.0,1.0,136.052,136.052,...,CC(=O)C1=CC=C(C=C1)O,CC(=O)C1=CC=C(C=C1)O,TXFPEBPIARQUIG-UHFFFAOYSA-N,1-(4-hydroxyphenyl)ethanone,2D+3D,NaN,Biological Test Results|Interactions and Pathw...,155|157|161|165|167|175|248|1576|1811|260086|2...,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...


In [12]:
# Function to convert SMILES to graph data
def smiles_to_graph(isosmiles):
    mol = Chem.MolFromSmiles(isosmiles)
    if mol is None:
        print(f"Invalid SMILES string: {isosmiles}")
        return None
    atom_features = [atom.GetAtomicNum() for atom in mol.GetAtoms()]

    edges = []
    for bond in mol.GetBonds():
        start_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        edges.append((start_idx, end_idx))
        edges.append((end_idx, start_idx))

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    x = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)

    return Data(x=x, edge_index=edge_index)

 

In [13]:
def train():
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch_graphs, labels = batch
        batch_graphs = batch_graphs.to(device)  # Directly move the batch to device
        labels = labels.to(device)
        optimizer.zero_grad()
        out = model(batch_graphs)
        loss = criterion(out, labels)  # Assuming binary classification with output in the second column
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_graphs.num_graphs
    return total_loss / len(train_dataset)

In [14]:
# Prepare data for training
graphs = []
labels = []

for _, row in df.iterrows():
    smiles = row['isosmiles']
    label = 1 if row['cmpdname'] == 'paracetamol' else 0  # Assuming 'paracetamol' is the target compound
    graph = smiles_to_graph(smiles)
    if graph is not None:
        graphs.append(graph)
        labels.append(label)

# Split dataset into training and validation sets
train_graphs, val_graphs, train_labels, val_labels = train_test_split(graphs, labels, test_size=0.2, random_state=42)

In [28]:
class GraphDataset(torch.utils.data.Dataset):
    def __init__(self, graphs, labels):
        self.graphs = graphs
        self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], torch.tensor(self.labels[idx], dtype=torch.float)

train_dataset = GraphDataset(train_graphs, train_labels)
val_dataset = GraphDataset(val_graphs, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [29]:
# Define a simple GCN model for binary classification
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(1, 64)
        self.conv2 = GCNConv(64, 1)  # Output layer for binary classification, output a single value
        self.pooling = global_mean_pool

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = self.pooling(x, data.batch)
        return torch.sigmoid(x).squeeze(1)  # Apply sigmoid for binary classification and squeeze to match target shape
 

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification

In [32]:
# Training and validation loop
num_epochs = 50
for epoch in range(num_epochs):
    train_loss = train()
    #val_accuracy = validate()
    #print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}') 
    # Validation Accuracy: {val_accuracy:.4f}')



Epoch 1, Train Loss: 0.0002
Epoch 2, Train Loss: 0.0001
Epoch 3, Train Loss: 0.0001
Epoch 4, Train Loss: 0.0001
Epoch 5, Train Loss: 0.0001
Epoch 6, Train Loss: 0.0001
Epoch 7, Train Loss: 0.0001
Epoch 8, Train Loss: 0.0001
Epoch 9, Train Loss: 0.0001
Epoch 10, Train Loss: 0.0001
Epoch 11, Train Loss: 0.0001
Epoch 12, Train Loss: 0.0001
Epoch 13, Train Loss: 0.0001
Epoch 14, Train Loss: 0.0001
Epoch 15, Train Loss: 0.0001
Epoch 16, Train Loss: 0.0001
Epoch 17, Train Loss: 0.0001
Epoch 18, Train Loss: 0.0001
Epoch 19, Train Loss: 0.0001
Epoch 20, Train Loss: 0.0001
Epoch 21, Train Loss: 0.0001
Epoch 22, Train Loss: 0.0001
Epoch 23, Train Loss: 0.0001
Epoch 24, Train Loss: 0.0001
Epoch 25, Train Loss: 0.0001
Epoch 26, Train Loss: 0.0001
Epoch 27, Train Loss: 0.0001
Epoch 28, Train Loss: 0.0001
Epoch 29, Train Loss: 0.0001
Epoch 30, Train Loss: 0.0001
Epoch 31, Train Loss: 0.0001
Epoch 32, Train Loss: 0.0001
Epoch 33, Train Loss: 0.0001
Epoch 34, Train Loss: 0.0001
Epoch 35, Train Loss: 0

In [41]:
def predict(smiles):
    model.eval()
    graph = smiles_to_graph(smiles)
    if graph is None:
        return "Invalid SMILES"
    
    graph = Batch.from_data_list([graph]).to(device)
    
    with torch.no_grad():
        out = model(graph)
        
        # If your model outputs logits for two classes, use softmax
        # probabilities = F.softmax(out, dim=1)
        
        # If your model outputs a single probability for the positive class
        if out.dim() == 1:  # Check if the output is 1D
            prob_paracetamol = out.item()  # Get the single probability
        else:
            prob_paracetamol = out[0, 1].item()  # Get the probability for the positive class

        return "Paracetamol" if prob_paracetamol > 0.5 else "Not Paracetamol"

# Test the prediction function
test_smiles = 'CC(=O)Nc1ccc(O)cc1C(=O)O'  # Example SMILES for paracetamol
print(f'Test SMILES: {test_smiles} => {predict(test_smiles)}')

Test SMILES: CC(=O)Nc1ccc(O)cc1C(=O)O => Not Paracetamol


In [42]:
import pickle

In [43]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)